# scraper Clasificados de los tiempos

notebook para obtenere datos de la página de los clasificados de los tiempos 

LINK: https://clasificados.lostiempos.com/inmuebles 

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import math
from datetime import datetime
from pyquery import PyQuery as pq

In [2]:
BASE_URL = "https://clasificados.lostiempos.com"
URL = f"{BASE_URL}/inmuebles?sort_by=created&sort_order=DESC&page="

HEADERS = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50"
                         ".0.2661.102 Safari/537.36"}
posts = []

In [3]:
response = requests.get(f"{URL}{0}", headers=HEADERS)
doc = pq(response.content)

In [4]:
def append_elements(doc, posts):
    l = doc.find('.views-row')
    for i, element in enumerate(l):
        data = get_data(l.eq(i))
        posts.append(data)

In [5]:
MONTHS_SPANISH = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]

# clean the text
def clean_text(s):
    return s.replace('\n', '').strip()

# get data from text like 'Publicado: 23 Marzo 2023'
def get_date(s):
    original_date = s.replace('Publicado: ', '')
    dt = original_date.split(' ')
    return f"{dt[2]}-{str(MONTHS_SPANISH.index(dt[1])).zfill(2)}-{dt[0].zfill(2)}"

# Get celphone and phone number 
def get_phones(s):
    return re.findall(r"([\d]{8}|[\d]{7})", s)

def get_price(s):
    # TODO: hacer el calculo de precios.
    pass

In [6]:
def get_data(element):
    title = element.children('.title')[0].text_content()
    date = element.children('.publish-date')[0].text_content()
    description = element.children('.body')[0].text_content()
    price = element.children('.ads-price')[0].text_content()
    keys = element.children('.description')[0].text_content()
    path = element.children('.buttons a')[0].attrib['href']

    return {
        'title': clean_text(title),
        'date': get_date(clean_text(date)),
        'description': clean_text(description),
        'price': clean_text(price),
        'keys': clean_text(keys),
        'path': clean_text(path),
    }

# get_data(doc.find('.views-row').eq(0))

In [7]:
doc.find('.views-row').eq(0).children('.buttons a')[0].attrib['href']

'/anuncio/inmuebles/departamento-en-venta-110/1983803'

In [8]:
number_elements = int(re.findall(r'\d+', doc.find('.resume-search')[0].text_content())[0])
NUMBER_OF_PAGES = math.ceil(number_elements / 20)
print("total:", number_elements, "- paginas:", NUMBER_OF_PAGES)

append_elements(doc, posts)

total: 69 - paginas: 4


In [9]:
# TEST if anything is wrong with errors
# posts_test = []
# append_elements(doc, posts_test)
# posts_test
# doc.find('.views-row').eq(0).children('.title')[0].text_content()

In [10]:
for n in range(1, NUMBER_OF_PAGES):
    print("get data from:", f"{URL}{n}")
    response = requests.get(f"{URL}{n}", headers=HEADERS)
    doc = pq(response.content)
    append_elements(doc, posts)

print("total de publicaciones:", len(posts))

get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=1
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=2
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=3
total de publicaciones: 69


In [11]:
df = pd.DataFrame(posts)
df

,title,date,description,price,keys,path
0,Departamento En Venta 110,2023-02-31,"DEPARTAMENTO En venta 110 m2, 3 dormitorios. ...",,Cochabamba - Venta - Departamento,/anuncio/inmuebles/departamento-en-venta-110/1...
1,Lotes Sipe Sipe Lote,2023-02-31,"LOTES Sipe Sipe, lote 1 4880 M2, lote 2 1811 ...",,Cochabamba - Venta - Lote / Terreno,/anuncio/inmuebles/lotes-sipe-sipe-lote/1983805
2,Doy Hermoso Dpto En,2023-02-31,Doy hermoso Dpto. en anticrético estilo Americ...,$us 27 500,Cochabamba - Anticrético - Departamento,/anuncio/inmuebles/doy-hermoso-dpto-en/1984002
3,Oficina Céntrica En Alquiler,2023-02-30,OFICINA céntrica en alquiler. 79380485.,,Cochabamba - Alquiler - Oficina,/anuncio/inmuebles/oficina-centrica-en-alquile...
4,Departamento Céntrico En Alquiler,2023-02-30,"DEPARTAMENTO céntrico, en alquiler. 70774733.",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/departamento-centrico-en-al...
...,...,...,...,...,...,...
64,1 800 Bs Mercado,2023-02-28,1.800 BS. Mercado 10 de Febrero Simón López de...,,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/1-800-bs-mercado/1983449
65,2 100 Bs Beijing,2023-02-28,2.100 BS. Beijing Tadeo Haenke Estrenar depart...,,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/2-100-bs-beijing/1983450
66,Casa En Anticrético Chalet,2023-02-28,"CASA En anticrético (chalet) 900 m2, construcc...",$us 80 000,Cochabamba - Anticrético - Casa / Chalet,/anuncio/inmuebles/casa-en-anticretico-chalet/...
67,Vendo Departamento Av Ballivián,2023-02-27,Vendo departamento Av Ballivián a pasos de pla...,,Cochabamba - Venta - Departamento,/anuncio/inmuebles/vendo-departamento-av-balli...


In [12]:
# Save dataframe as .csv file 
df.to_csv(f'./csv/los_tiempos_clasificados_{datetime.now().isoformat()[:10]}.csv', sep=';')

In [13]:
!pip freeze

pandas==1.4.4
pyquery==2.0.0
requests==2.28.2

SyntaxError: invalid syntax (1479320003.py, line 3)